# Load List of Kings and Queens
Load from xlsx because:
* CSVs are corrupt
* API from Parlamentsdienste is a pain
Download at the bottom: https://www.zh.ch/de/politik-staat/wahlen-abstimmungen/kantons-regierungsratswahlen/mitglieder-kantonsrats-ab-1803.html#-373901206

In [27]:
import pandas as pd
from datetime import datetime
from pathlib import Path
import numpy as np
import json

In [6]:
df_person = pd.read_excel(Path('../data/kantonsrathistory.xlsx'), 'Personen')
df_funktion = pd.read_excel(Path('../data/kantonsrathistory.xlsx'), 'Funktionen')
df_partei = pd.read_excel(Path('../data/kantonsrathistory.xlsx'), 'Parteien')
df_einsitz = pd.read_excel(Path('../data/kantonsrathistory.xlsx'), 'Einsitze')

In [43]:
df_person = df_person[df_person.DATUM_GEBURT_JAHR.notna()]
df_person = df_person[df_person.DATUM_GEBURT_JAHR> 1900]
df_einsitz = df_einsitz[df_einsitz.DATUM_EINTRITT_JAHR.notna()]
df_funktion = df_funktion[df_funktion.DATUM_VON_JAHR.notna()]
df_funktion = df_funktion[df_funktion.DATUM_BIS_JAHR >= 2000]
df_partei = df_partei[df_partei.PARTEIBEZEICHNUNG.notna()]

records = []

for i, row in df_person.iterrows():

    record = {
        'id': row['ID_PERSON_NEW'],
        'name': row['NACHNAME'],
        'vorname': row['VORNAME'],
        'geschlecht': row['GESCHLECHT'],
        'jahrgang': row['DATUM_GEBURT_JAHR'],
        'einsitz': [],
        'partei': [],
        'funktion': []
    }

    # Add Einsitz
    df = df_einsitz[df_einsitz.ID_PERSON_NEW == row['ID_PERSON_NEW']]
    for y, row2 in df.iterrows():
        if np.isnan(row2['DATUM_AUSTRITT_JAHR']):
            end = datetime(2100, 12, 31)
        else:
            end = datetime(int(row2['DATUM_AUSTRITT_JAHR']), int(row2['DATUM_AUSTRITT_MONAT']), int(row2['DATUM_AUSTRITT_TAG']))

        record['einsitz'].append({
            'start': datetime(int(row2['DATUM_EINTRITT_JAHR']), int(row2['DATUM_EINTRITT_MONAT']), int(row2['DATUM_EINTRITT_TAG'])),
            'end': end
        })

    # Add Party
    df = df_partei[df_partei.ID_PERSON_NEW == row['ID_PERSON_NEW']]
    try:
        for y, row2 in df.iterrows():
            if not(np.isnan(row2['DATUM_BIS_MONAT'])) and not(np.isnan(row2['DATUM_BIS_JAHR'])):
                end = datetime(int(row2['DATUM_BIS_JAHR']), int(row2['DATUM_BIS_MONAT']), int(row2['DATUM_BIS_TAG']))
            else:
                end = datetime(2100, 12, 31)

            if np.isnan(row2['DATUM_VON_JAHR']):
                start = record['einsitz'][0]['start']
            else:
                datetime(int(row2['DATUM_VON_JAHR']), int(row2['DATUM_VON_MONAT']), int(row2['DATUM_VON_TAG']))

            record['partei'].append({
                'bezeichnung': row2['PARTEIBEZEICHNUNG'],
                'start': start,
                'end': end
            })
    except:
        print(row2)
        raise
        
    # Add Funktion
    df = df_funktion[df_funktion.ID_PERSON_NEW == row['ID_PERSON_NEW']]
    try:
        for y, row2 in df.iterrows():
            # Assume, that Amtsjahr starts at 1. Mai and ends at 30. April
            if not np.isnan(row2['DATUM_VON_JAHR']):
                day = 1 if np.isnan(row2['DATUM_VON_TAG']) else int(row2['DATUM_VON_TAG'])
                month = 5 if np.isnan(row2['DATUM_VON_MONAT']) else int(row2['DATUM_VON_MONAT'])
                start = datetime(int(row2['DATUM_VON_JAHR']), month, day)
            else:
                start = datetime(2100, 12, 31)
                print("Hmmm")


            if np.isnan(row2['DATUM_BIS_JAHR']):
                end = datetime(2100, 12, 31)
            else:
                day = 30 if np.isnan(row2['DATUM_BIS_TAG']) else int(row2['DATUM_BIS_TAG'])
                month = 4 if np.isnan(row2['DATUM_BIS_MONAT']) else int(row2['DATUM_BIS_MONAT'])
                end = datetime(int(row2['DATUM_BIS_JAHR']), month, day)                

            record['funktion'].append({
                'bezeichnung': row2['FUNKTIONSBEZEICHNUNG'],
                'start': start,
                'end': end
            })

    except:
        print(row2)
        raise
    records.append(record)

    #break

In [48]:
with open(Path('../export/mitglieder.json'), 'w') as f:
    json.dump(records, f, default=str)

DEPRICATED

In [2]:
df_person = pd.read_excel(Path('../data/kantonsrathistory.xlsx'), 'Personen')
df_funktion = pd.read_excel(Path('../data/kantonsrathistory.xlsx'), 'Funktionen')

# Take newest Party. Could be improved
df_partei = pd.read_excel(Path('../data/kantonsrathistory.xlsx'), 'Parteien')
df_partei = df_partei.sort_values('DATUM_VON_JAHR', ascending=False)
df_partei = df_partei.drop_duplicates(['ID_PERSON_NEW'], keep='first')

In [3]:
df = df_funktion.merge(df_person, on = 'ID_PERSON_NEW')
df = df.merge(df_partei[['ID_PERSON_NEW', 'PARTEIBEZEICHNUNG']], on = 'ID_PERSON_NEW')

df = df[(df.DATUM_VON_JAHR >= 1980) | (df.DATUM_VON_JAHR.isna())]


records = []
for i, row in df.iterrows():

    try:
        if not np.isnan(row['DATUM_VON_JAHR']) and not(np.isnan(row['DATUM_VON_MONAT'])):
            start = datetime(int(row['DATUM_VON_JAHR']), int(row['DATUM_VON_MONAT']), int(row['DATUM_VON_TAG']))
        else:
            start = datetime(1700, 1, 1)

        if (not np.isnan(row['DATUM_BIS_JAHR'])) and (not np.isnan(row['DATUM_BIS_MONAT'])):
            end = datetime(int(row['DATUM_BIS_JAHR']), int(row['DATUM_BIS_MONAT']), int(row['DATUM_BIS_TAG']))
        else:
            end = datetime(9999, 12, 31, 23, 59, 59)
    except Exception as e:
        print(e)
        print(row)
        raise

    record = {
        'id': row['ID_PERSON_NEW'],
        'name': row['NACHNAME'],
        'vorname': row['VORNAME'],
        'geschlecht': row['GESCHLECHT'],
        'jahrgang': row['DATUM_GEBURT_JAHR'],
        'partei': row['PARTEIBEZEICHNUNG'],
        'funktion': row['FUNKTIONSBEZEICHNUNG'],
        'start': datetime.strftime(start, '%Y-%m-%dT00:00:00.000'),
        'end': datetime.strftime(end, '%Y-%m-%dT00:00:00.000'),
    }
    records.append(record)

df_res = pd.DataFrame(records)
df_res.to_csv(Path('../export/mitglieder.csv'), index=False)

print("finito")

finito
